# MGMTMSA 403 Assignment 3: Predicting Airbnb Prices

#### By Xiaoyi Wang

In [26]:
import numpy as np
import pandas as pd
from gurobipy import *

train = pd.read_csv("AirbnbTrain.csv")
test = pd.read_csv("AirbnbTest.csv")

In [42]:
train = train[~train.isin(['Nan']).any(1)].reset_index(drop=True)
test = test[~test.isin(['Nan']).any(1)].reset_index(drop=True)

In [168]:
vnum = len(train.columns) -1
cnum= len(train)
tnum = len(test)
p = list(train[train.columns[-1]])
tp = list(test[test.columns[-1]])

tr = train.values.tolist()
te = test.values.tolist()

#Model3
bed_index = train.columns.get_loc("beds")

mod = Model()
#Model1,2,3
b = mod.addVars(vnum)
#Model2
l = mod.addVars(vnum, vtype = GRB.BINARY)

#constraints
for y in range(cnum):
    #Model1,2,3
    mod.addConstr((sum(b[x] * tr[y][x] for x in range(vnum)) - p[y]) >= 0)

#Model 2
limit_con = mod.addConstr(sum(l[x] for x in range(vnum)) == 3)
for x in range(vnum):
    mod.addConstr(b[x] * l[x] == b[x])

#Model 3
mod.addConstr(l[bed_index] == 1)

mod.setObjective(sum(sum(b[x] * tr[y][x] for x in range(vnum)) - p[y] for y in range(cnum)), GRB.MINIMIZE)
mod.update()
mod.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1700 rows, 24 columns and 18977 nonzeros
Model fingerprint: 0x6a32346e
Model has 12 quadratic constraints
Variable types: 12 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+03, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 414 rows and 1 columns
Presolve time: 0.01s
Presolved: 1319 rows, 56 columns, 14410 nonzeros
Presolved model has 22 SOS constraint(s)
Variable types: 34 continuous, 22 integer (22 binary)

Root relaxation: objective 1.374383e+06, 15 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1374382.89    0    

#### Model 1:

In [158]:
def PredEr(b):
    result = 0
    for x in range(vnum):
        for y in range(tnum):
            s = 0
            s += b[x].X * te[y][x]
        result += (s - tp[x])
    result = result * (1 / vnum)
    return result
print('the prediction error is:' , "{:.2f}".format(PredEr(b)), '$/night')

the prediction error is: -67.46 $/night


#### Model 2:

In [162]:
def parameters(b):
    parameters = []
    for x in range(vnum):
        if b[x].X != 0:
            parameters.append(b[x].VarName)
            
    for x in range(len(parameters)):
        parameters[x] = parameters[x][1:]
        parameters[x] = test.columns[int(parameters[x])]
    return parameters

def coef(b):
    coef = []
    for x in range(vnum):
        if b[x].X != 0:
            coef.append("{:.7f}".format(b[x].X))
    return coef
print('the prediction error is:' , "{:.2f}".format(PredEr(b)), '$/night')
print('the parameters used are:' , parameters(b))
print('the coefs of parame are:' , coef(b))

the prediction error is: -50.75 $/night
the parameters used are: ['longitude', 'bathrooms', 'instant_bookable']
the coefs of parame are: ['2.8228811', '778.0350216', '556.0070596']


#### Model 3:

In [169]:
print('the prediction error is:' , "{:.2f}".format(PredEr(b)), '$/night')
print('the parameters used are:' , parameters(b))
print('the coefs of parame are:' , coef(b))

the prediction error is: -50.75 $/night
the parameters used are: ['bathrooms', 'instant_bookable']
the coefs of parame are: ['666.6666667', '333.3333333']


Longitude is no longer in the list which might be the coeficient is to low to have an impact on the prediction. It only exists in model 2 only because we force the model to pick 3 variables. Apparently 2 are enough to achieve a lower prediction error.